## 问答题
1. 支持向量机的基本思想是什么？

2. 什么是支持向量？

3. 在使用 SVM 时，缩放输入值为什么很重要？

4. SVM 分类器在对实例进行分类时能输出置信度分数吗？概率呢？

5. 你如何在 LinearSVC、SVC 和 SGDClassifier 之间进行选择？

6. 假设你已经使用 RBF 核训练了一个 SVM 分类器，但它似乎欠拟合训练集。
   你应该增大还是减小 γ（gamma）？C 呢？

7. ε 不敏感模型是什么意思？

8. 使用核技巧有什么意义？

## 编程题
1. 在葡萄酒数据集上训练SVM分类器，可以使用sklearn.datasets.load_wine()加载它。该数据集包含3个不同种植者生产的178个葡萄酒样本的化学分析：目标是训练一个分类模型，该模型能够根据葡萄酒的化学分析预测种植者。由于SVM分类器是二元分类器，将需要使用“一对全部”对所有三个类进行分类。能达到的精度是多少？

   "一对全部"可以复习 **8_sklearn做分类.ipynb**里的笔记，里面提到了用二元分类器做多分类问题

---

2. 提前预习 **10_支持向量机.ipynb** 最新更新的笔记 （把SVM分类用梯度下降实现）； 大概理解笔记后，尝试自己对照笔记 实现用梯度下降实现SVM分类

   并把自定义的SVM分类用于 iris data(鸢尾花数据)； 取花瓣长度 和 花瓣宽度特征， 分类 看是不是 分类2的花 （(iris.target == 2)

   对比下sklearn自带的SVM分类 和 自定义SVM分类 实现的分类效果

---

3. 在加州房屋数据集上训练和微调SVM回归器。可以使用原始数据集而不是 在课上使用的调整后的版本，
可以使用sklearn.datasets.fetch_california_housing()加载它。目标代表了数十万美元。
由于有超过20000个实例，SVM可能会很慢，因此对于超参数调整，应该使用更少的实例（例如2000个）来测试更多的超参数组合。最佳模型的RMSE是多少？


In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform

# 加载加州房屋数据集
housing = fetch_california_housing()
X, y = housing.data, housing.target
print(f"数据集规模: {X.shape[0]}个实例, {X.shape[1]}个特征")
print(f"目标值范围: {y.min():.2f} - {y.max():.2f} (十万美元)")

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 创建小规模数据集用于超参数搜索（2000个实例）
X_train_small = X_train[:2000]
y_train_small = y_train[:2000]
print(f"用于超参数调优的小规模数据集: {X_train_small.shape[0]}个实例")

# 创建管道：先标准化（SVM对特征尺度敏感），再SVM回归
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

# 定义超参数搜索空间
param_dist = {
    'svr__kernel': ['linear', 'rbf', 'poly'],
    'svr__C': loguniform(1e-3, 1e3),  # 正则化参数，较大的值表示较少的正则化
    'svr__gamma': ['scale', 'auto'] + list(loguniform(1e-4, 1e2).rvs(5)),  # 核系数
    'svr__epsilon': loguniform(1e-2, 1e0)  # 不敏感区域大小
}

# 随机搜索超参数（比网格搜索更高效）
random_search = RandomizedSearchCV(
    pipeline,
    param_dist,
    n_iter=30,  # 尝试30种参数组合
    cv=5,       # 5折交叉验证
    scoring='neg_mean_squared_error',
    n_jobs=-1,  # 使用所有可用CPU
    random_state=42,
    verbose=1
)

# 在小规模数据集上执行超参数搜索
print("\n开始超参数搜索...")
random_search.fit(X_train_small, y_train_small)

# 输出最佳参数
print("\n最佳超参数:", random_search.best_params_)

# 使用最佳参数在完整训练集上训练模型
print("\n使用最佳参数在完整训练集上训练...")
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train)

# 在测试集上评估模型
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\n最佳模型在测试集上的RMSE: {rmse:.4f} (十万美元)")
print(f"最佳模型在测试集上的RMSE: {rmse * 100:.2f} (千美元)")
